# Cleaning and Feature Engineering

---

This notebook focuses on the cleaning and feature engineering steps in preparing the dataset for Ethereum price prediction. Key processes include lagging the closing prices, calculating daily price changes (dayChange), measuring volatility, and creating additional features such as 'perc75_Neg' to identify addresses with better predictive capabilities.

#### 1. Lagging Closing Prices

   - Lag the closing price of Ethereum for the past 4 days.

#### 2. Calculating DayChange

   - Measure the daily change in Ethereum closing prices.

#### 3. Calculating Volatility

   - Compute volatility, representing the variance over the past 4 days.

#### 4. Creating perc75_Neg Columns

   - Identify addresses with high volatility in the top 75 percentile and predict future changes.

#### 5. Mapping Information to Transactions

   - Map the engineered features onto each Ethereum transaction.

#### 6. Mapping Transaction Frequency

   -  Map transaction frequency onto the price per day dataframe.

### Imports

In [205]:
import pandas as pd
import numpy as np
from datetime import datetime

### Data Read in


Price of Ethereum since being tracked by Yahoo finance, I beleive this might also be able to decrease irregularities of the start of the cryptocurrency.

In [206]:
price_data = pd.read_csv('../data/eth_price_per_day.csv')
track_start = int(datetime.strptime(price_data['Date'][0], "%Y-%m-%d").timestamp())
track_end =int(datetime.strptime(price_data['Date'].iloc[-1], "%Y-%m-%d").timestamp())
price_data['Date'] = pd.to_datetime(price_data['Date'])
price_data.set_index('Date', inplace=True)
price_data.sort_index(ascending=False, inplace=True)
price_data.head(3)

,Close
Date,
2024-01-31,2282.544434
2024-01-30,2344.493652
2024-01-29,2317.064209


Looking at only data after Yahoo began tracking so that we can compare volatility.

In [207]:
eth_data1 = pd.read_csv('../data/eth_trans_data1.csv', index_col=None)
eth_data2 = pd.read_csv('../data/eth_trans_data2.csv', index_col=None)

eth_data = pd.concat([eth_data1, eth_data2], axis=0)

eth_data.drop(columns='Unnamed: 0', inplace=True)

eth_data['value'] = eth_data['value'].astype(float)

C:\Users\mcdun\AppData\Local\Temp\ipykernel_4200\3873121584.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_data2 = pd.read_csv('../data/eth_trans_data2.csv', index_col=None)


## Feature Engineering Price of Ethereum Data
----
Calculated volatility with recommendation from Corporate Finance Institute
https://corporatefinanceinstitute.com/resources/career-map/sell-side/capital-markets/volatility-vol/


In [208]:
display(price_data.head())
price_data.info()

,Close
Date,
2024-01-31,2282.544434
2024-01-30,2344.493652
2024-01-29,2317.064209
2024-01-28,2257.208740
2024-01-27,2267.885986


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2275 entries, 2024-01-31 to 2017-11-09
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2275 non-null   float64
dtypes: float64(1)
memory usage: 35.5 KB


Creating change in price and volatility columns

In [209]:
price_data['Close_1']=price_data['Close'].shift(1)
price_data['Close_2']=price_data['Close'].shift(2)
price_data['Close_3']=price_data['Close'].shift(3)
price_data['Close_4']=price_data['Close'].shift(4)

avg_last4 = (price_data['Close']+price_data['Close_1']+price_data['Close_2']+price_data['Close_3']+price_data['Close_4'])/5

diff = price_data['Close'] - avg_last4
diff1 = price_data['Close_1'] - avg_last4
diff2 = price_data['Close_2'] - avg_last4
diff3 = price_data['Close_3'] - avg_last4
diff4 = price_data['Close_4'] - avg_last4

price_data['volatility']= ((diff**2)+(diff1**2)+(diff2**2)+(diff3**2)+(diff4**2)/5)**0.5

price_data['dayChange'] = price_data['Close'] - price_data['Close_1']
price_data.head(10)

,Close,Close_1,Close_2,Close_3,Close_4,volatility,dayChange
Date,,,,,,,
2024-01-31,2282.544434,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-30,2344.493652,2282.544434,NaN,NaN,NaN,NaN,61.949219
2024-01-29,2317.064209,2344.493652,2282.544434,NaN,NaN,NaN,-27.429443
2024-01-28,2257.208740,2317.064209,2344.493652,2282.544434,NaN,NaN,-59.855469
2024-01-27,2267.885986,2257.208740,2317.064209,2344.493652,2282.544434,71.736637,10.677246
2024-01-26,2267.199707,2267.885986,2257.208740,2317.064209,2344.493652,58.942905,-0.686279
2024-01-25,2217.710205,2267.199707,2267.885986,2257.208740,2317.064209,53.719713,-49.489502
2024-01-24,2233.561768,2217.710205,2267.199707,2267.885986,2257.208740,43.755391,15.851562
2024-01-23,2240.686035,2233.561768,2217.710205,2267.199707,2267.885986,38.804060,7.124268


Creation of Volatility lags per day columns

In [210]:
price_data.dropna(inplace=True)
price_data['perc75_Neg'] = ((price_data['volatility'] > 109) & (price_data['dayChange'] < 0))
price_data['perc75_Neg'] = price_data['perc75_Neg'].astype(int)
price_data['perc75_Neg_lag1'] = price_data['perc75_Neg'].shift(1)
price_data['perc75_Neg_lag2'] = price_data['perc75_Neg'].shift(2)
price_data['perc75_Neg_lag3'] = price_data['perc75_Neg'].shift(3)
price_data['get_out_metric'] = price_data['perc75_Neg_lag1']+price_data['perc75_Neg_lag2']+price_data['perc75_Neg_lag3']
price_data.fillna(0, inplace=True)
price_data.head(10)

,Close,Close_1,Close_2,Close_3,Close_4,volatility,dayChange,perc75_Neg,perc75_Neg_lag1,perc75_Neg_lag2,perc75_Neg_lag3,get_out_metric
Date,,,,,,,,,,,,
2024-01-27,2267.885986,2257.208740,2317.064209,2344.493652,2282.544434,71.736637,10.677246,0,0.0,0.0,0.0,0.0
2024-01-26,2267.199707,2267.885986,2257.208740,2317.064209,2344.493652,58.942905,-0.686279,0,0.0,0.0,0.0,0.0
2024-01-25,2217.710205,2267.199707,2267.885986,2257.208740,2317.064209,53.719713,-49.489502,0,0.0,0.0,0.0,0.0
2024-01-24,2233.561768,2217.710205,2267.199707,2267.885986,2257.208740,43.755391,15.851562,0,0.0,0.0,0.0,0.0
2024-01-23,2240.686035,2233.561768,2217.710205,2267.199707,2267.885986,38.804060,7.124268,0,0.0,0.0,0.0,0.0
2024-01-22,2310.826416,2240.686035,2233.561768,2217.710205,2267.199707,71.944097,70.140381,0,0.0,0.0,0.0,0.0
2024-01-21,2453.913086,2310.826416,2240.686035,2233.561768,2217.710205,183.842881,143.086670,0,0.0,0.0,0.0,0.0
2024-01-20,2469.589111,2453.913086,2310.826416,2240.686035,2233.561768,206.010996,15.676025,0,0.0,0.0,0.0,0.0
2024-01-19,2489.498535,2469.589111,2453.913086,2310.826416,2240.686035,174.081425,19.909424,0,0.0,0.0,0.0,0.0


### Feature Engineering of Transaction Data
---

In [211]:
eth_data['contractAddress'].value_counts()

contractAddress
0x5630a44404df787ec75105bd97e64bea587344ab    2
0x9a049f5d18c239efaa258af9f3e7002949a977a0    2
0xc250a22436cdfbe47c0b8ca32a5834932964d1cb    2
0xe6f13c5af166f8c732873ae069f53fa4cbf1230f    2
0x2963fb5fb6381f4961668af9a42d23750944e682    2
0x3e75dc35798ef753f856d501c3d610057078e6a5    2
0xa1162cbb7f6cc8f8476c5f4783761302a9abaf69    2
0xf022a69b94fe837f53d2fa1891313aeea22a52be    2
0x94f27b5141e17dd8816242d752c7be8e6764bd22    2
0xc9fb84ecf23ed92eb8d24264be79909546f67889    2
0x283af0b28c62c092c9727f1ee09c02ca627eb7f5    2
0x63520acb8ca0532993c46d7077f32074287b9e92    2
0x3e74478185e4a740228da645140127a52f707594    2
0x96bf6cf4a2528851d27220b74c45258e7dcf5a73    2
0x708954b36410f91573fb0dbd0a2d9158ff53578a    2
0x20d60ec8a9090b3c939e47390cda758ed0086637    2
0x122d06a722f3ee4afa33d3b19aba0671bfc98581    2
0xd91efec7e42f80156d1d9f660a69847188950747    2
0xc2c925e7128896e4ae33e354602c70694a0088cd    2
0x85cb4fab90bf67e2593891cbe75385eff0067188    2
0xd29dfe5ae95b5c067a91f4

In [212]:
eth_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 269499 entries, 0 to 176958
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   blockNumber        269499 non-null  int64  
 1   timeStamp          269499 non-null  int64  
 2   hash               269499 non-null  object 
 3   nonce              269495 non-null  float64
 4   blockHash          269495 non-null  object 
 5   transactionIndex   269499 non-null  int64  
 6   from               269499 non-null  object 
 7   to                 269419 non-null  object 
 8   value              269499 non-null  float64
 9   gas                269499 non-null  int64  
 10  gasPrice           269499 non-null  int64  
 11  isError            269499 non-null  int64  
 12  txreceipt_status   266595 non-null  float64
 13  input              269495 non-null  object 
 14  contractAddress    80 non-null      object 
 15  cumulativeGasUsed  269499 non-null  int64  
 16  gasUsed

In [213]:
# Most transactions do not use a contract.
eth_data['contractAddress'].fillna('no_contract', inplace=True)

# Creating easy to use time Columns 
eth_data['dateTime'] = pd.to_datetime(eth_data['timeStamp'], unit='s')
eth_data['timeOnly'] = eth_data['dateTime'].dt.time
eth_data['dateOnly'] = eth_data['dateTime'].dt.date
eth_data['hoursOftheday'] = eth_data['timeOnly'].apply(lambda x: x.hour)
eth_data['dateOnly'] = pd.to_datetime(eth_data['dateOnly'])
eth_data.head(3)

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,dateTime,timeOnly,dateOnly,hoursOftheday
0,50937,1438996872,0xf32063c80f932845cb3eff4ef1c5e2c641c09e80e20f...,6.0,0xae9906ff37bffbc90ea971c18de8c2664b7b6319b38b...,0,0xc8ebccc5f5689fa8659d83713341e5ad19349448,0x0f87fb507513e749081e34373c84e95e85b5c057,1.000000e+21,90000,...,no_contract,21000,21000,19171628,0x,NaN,2015-08-08 01:21:12,01:21:12,2015-08-08,1
1,68803,1439294010,0x60a38754856bffe131caa33526265918a5baa218f3f1...,48.0,0xa4a2c18349acf0b9729fd1c159714f56ba4bce0f2e44...,0,0xc8ebccc5f5689fa8659d83713341e5ad19349448,0x0f87fb507513e749081e34373c84e95e85b5c057,2.000000e+20,90000,...,no_contract,21000,21000,19153762,0x,NaN,2015-08-11 11:53:30,11:53:30,2015-08-11,11
2,73182,1439364508,0x6532654ed29dece6cd0080ffcf1196ce3e183727ee5b...,50.0,0xbd52d10a10e683623a2ccb49d915f2724c35f11e793f...,0,0xc8ebccc5f5689fa8659d83713341e5ad19349448,0x0f87fb507513e749081e34373c84e95e85b5c057,7.700000e+19,90000,...,no_contract,21000,21000,19149383,0x,NaN,2015-08-12 07:28:28,07:28:28,2015-08-12,7


In [215]:
eth_data['ethValusd'] = eth_data['dateOnly'].map(price_data['Close'])
eth_data['volatility'] = eth_data['dateOnly'].map(price_data['volatility'])
eth_data['dayChange'] = eth_data['dateOnly'].map(price_data['dayChange'])
eth_data['perc75_Neg'] = eth_data['dateOnly'].map(price_data['perc75_Neg'])

eth_data['perc75_Neg_lag1'] = eth_data['dateOnly'].map(price_data['perc75_Neg_lag1'])
eth_data['perc75_Neg_lag2'] = eth_data['dateOnly'].map(price_data['perc75_Neg_lag2'])
eth_data['perc75_Neg_lag3'] = eth_data['dateOnly'].map(price_data['perc75_Neg_lag3'])
eth_data['get_out_metric'] = eth_data['dateOnly'].map(price_data['get_out_metric'])


eth_data['valueUSD'] = eth_data['value']*eth_data['ethValusd']/(1000000000000000000)

##nulls due to mismatch timeline may be dropped at a later point.

In [216]:
eth_data.drop(columns=['gas','txreceipt_status'], inplace=True) ## not needed

In [217]:
eth_data.to_csv('../data/eth_trans_data_clean.csv', index=False)

### Price Per Day Feature Engineering
---
Needed to be cleaned first.

In [219]:
transaction_count = eth_data.groupby(['from', 'dateOnly']).size().reset_index(name='transactions_count')
transaction_count.set_index('dateOnly', inplace=True)

transactions_per_day = pd.pivot_table(transaction_count, values='transactions_count', index='dateOnly', columns='from', aggfunc='sum', fill_value=0)

merged_data = pd.merge(price_data, transactions_per_day, left_index=True, right_index=True, how='left')
merged_data.fillna(0,inplace=True)

In [220]:
merged_data.to_csv('../data/price_data_merged_w_trans_count.csv')